In [21]:
import re
import os

changes = {}
sectchanges = {}
madechanges = {}

standardizations = {}
standardizations['pre-eminent'] = 'preeminent'
def standardizevoc(s):
    for foo in standardizations:
        
        s = re.sub(foo,standardizations[foo],s)
        s = re.sub('--','—',s)
        #s = re.sub('Chaldean','Chaldaean',s)
        s = re.sub('practic([ie])','practis\g<1>',s)
        s = re.sub('endeavor','endeavour',s)
        s = re.sub('([tT]he word|called|appellation)\s+"\s*([^\s]+)\s*"','\g<1> "\g<2>"',s)

        s = re.sub('([Pp])ractis','\g<1>ractic',s)
        s = re.sub('—',' — ',s)
        s = re.sub('\s+([,;:!?])','\g<1>',s)
    return(s)

donotsub = ['24.2.163','24.2.177','24.1.194','24.2.183','24.2.194','24.2.199','24.2.200','24.2.205','24.2.207']

totwords = 0
totchanges = 0
def compvers(workn):
    curwords = 0
    prevsect = 0
    global changes
    global sectchanges
    global madechanges
    global donotsub
    global totwords
    global totchanges
    madechanges = {}
    changes = {}
    sectchanges = {}
    
    
    outfname1 = '/Users/gcrane/scratch/lucian/fowler-sac/words' + workn + '.txt'

    
    
    infname = '/Users/gcrane/github/philoalex/yonge/works/tlg0018.tlg0' + workn + '.yonge_eng1.xml'
    infname = '/Users/gcrane/github/summer2023/lucian/fowlers/lucian-vol' + workn + '-eng.xml'
    
    outfname2 = 'words2-'+workn+'.txt'
    outf = open(outfname2,'w')
    f = open(infname)
    cursect = ''
    curbook = ''
    curpage = ''
    for l in f:
        l = re.sub('\s+$','',l)
        
        m = re.search('<pb n="([^>]+)"',l)
        if(m):
            curpage = m[1]
        m = re.search('type="section" n="([0-9]+)"',l)
        if(m):
            cursect = m[1]
        m = re.search('type="book" n="([0-9]+)"',l)
        if(m):
            curbook = m[1] + '.'
        
        if(not l):
            continue
#        if(not cursect):
        if(not curpage):
            continue
          
        l = standardizevoc(l)
        l = re.sub('(<label>[^<]+)\s+','\g<1>_',l)
        l = re.sub('(<label>[^<]+)\s+','\g<1>_',l)
        l = re.sub('<label>([^<]+)</label>','@\g<1>',l)
        l = re.sub('<bibl[^>]*>[^<]+</bibl>',' ',l)
        l = re.sub('<note[^>]*>[^<]+</note>','',l)
        l = re.sub('<note[^>]*><p>.+</p></note>','',l)
        l = re.sub('<head[^>]*>[^<]+</head>',' ',l)
        l = re.sub('</(title|q)>','',l)
        l = re.sub('<[^>]+>',' ',l)
        l = re.sub(' [FH]\.',' ',l)
        l = re.sub('’',"'",l)
        l = re.sub('‘',"'",l)
        
        for foo in l.split():
            #print(curbook+cursect,foo,file=outf)
            print(curpage,foo,file=outf)
            curwords = curwords + 1
            
    
    f.close()
    outf.close()
    diff_file = 'engdiff-' + workn + '.txt'

    comm = 'diff -y ' + outfname1 + ' ' + outfname2 + " > " + diff_file
    print(comm)
    os.system(comm) 
    
    
    f = open(diff_file)
    comps = []
    for l in f:
        l = re.sub('\s+$','',l)
        words = l.split()
        comps.append(words)
        
    for i in range(0,len(comps)-1):
        #print(words,len(words))
        words = comps[i]
        #print('words',words)
        skipkey = workn + '.' + words[0]
        if(skipkey in donotsub):
            continue
        if(re.search('(@|nbsp;)',l)):
            continue
        if(len(words)== 5 and words[2] == '|'):
            word1a = words[1]
            word2a = words[4]
            if(re.search('([α-ω]|<foreign|foreign>|@)',word2a)):
                continue
            #print(word1a,word2a,words)
            nextwords = comps[i+1]
            if(nextwords[2] == '<'):
                word1b = nextwords[1]
                startval = word2a
                endval = word1a + ' ' + word1b
                #print('a',words[0],startval,endval)
                
            elif(nextwords[0] == '>' and not comps[i+2][0] == '>'):
                word2b = nextwords[2]
                startval = word2a + ' ' + word2b
                endval = word1a
                #print('b',words[0],startval,endval)
            elif(words[2] == '|'):
                startval = words[4]
                endval = words[1]
                if(re.search('-',startval) and (re.sub('([a-z])-([a-z])','\g<1>\g<2>',startval) == endval)):
                    #print('keep hyphen!',words[0],startval,endval)
                    continue
                if( startval == endval.lower()):
                    #print('keep case!',words[0],startval,endval)
                    continue
                #print('c',words[0],startval,endval)
            if(comps[i-1][0] == comps[i][0] and not comps[i-1][2] in ['|','<','>']):
                startval = comps[i-1][3] + ' ' + startval
                endval = comps[i-1][1]  + ' ' + endval
                #print('now',startval,'--',endval)
            elif(comps[i+1][0] == comps[i][0] and not comps[i+1][2] in ['|','<','>']):
                startval = startval + ' ' + comps[i+1][3]
                endval = endval + ' ' + comps[i+1][1]
                #print('now',startval,'--',endval)
            curkey = words[0] + ' ' + startval
            # the following was useful for Philo where we had many falsely upper case words
            if(0 and startval.lower() == startval and not re.search('\\bI\\b',endval)):
                
                changes[curkey] = endval.lower()
            else:
                changes[curkey] = endval
            if(not words[0] in sectchanges):
                sectchanges[words[0]] = []
            sectchanges[words[0]].append(startval)
            
        
        
    f.close()
    
    infname = '/Users/gcrane/github/philoalex/yonge/works/tlg0018.tlg0' + workn + '.yonge_eng1.xml'
    infname = '/Users/gcrane/github/summer2023/lucian/fowlers/lucian-vol'+workn+'-eng.xml'
    outfname3 = re.sub('lucian-vol','corr/corr-lucian-vol',infname)
    outf = open(outfname3,'w')
    f = open(infname)
    cursect = ''
    curbook = ''
    curpage = ''
    for l in f:
        l = re.sub('\s+$','',l)
        l = re.sub('<note type="head">[^<]+</note>\s*','',l)
        l = standardizevoc(l)
        m = re.search('type="book" n="([0-9]+)"',l)
        if(m):
            curbook = m[1] + '.'
        m = re.search('type="section" n="([0-9]+)"',l)
        if(m):
            cursect = m[1]
        m = re.search('<pb n="([0-9]+)"',l)
        if(m):
            curpage = m[1]
            
        if(not cursect or re.search('<notex',l)):
            print(l,file=outf)
            continue
            
        skipkey = workn + '.' + curbook + cursect
        booksect = curbook+cursect
        booksect = curpage
        if(booksect in sectchanges and not skipkey in donotsub):
            for foo in sectchanges[booksect]:
                curkey = booksect + ' ' + foo
                 
                if(curkey in changes):
                    startval = foo
                    if(re.search('^[A-Za-z]',startval)):
                        startval = '\\b' + startval
                    if(re.search('[a-z]$',startval)):
                        startval =  startval + '\\b'
                    startval = re.sub('(\||\(|\))','.',startval)
                    startval = re.sub('(\?)','\\?',startval)
                    startval = re.sub('\.','\\.',startval)
                    if(re.search("zzzAnd 'how",foo)):
                        print("And 'how1",foo,startval)
                    if(re.search("'",startval)):
                        #print('s1',startval)
                        startval = re.sub("'","[’‘'‘]",startval)
                        #print('s2',startval)
                    if(re.search("zzzAnd 'how",foo)):
                        print("And 'how2",foo,startval)

                    #startval = re.sub('^([a-zA-Z])','\\b'+'\g<1>',foo)
                    #startval = re.sub('([a-zA-Z])$','\g<1>'+'\\b',startval)
                    #startval = re.sub('([\\\|\.\(\)])','\\\\'+'\g<1>',startval)
                    #print('testing',curkey,startval,changes[curkey],l,sep='\t')
                    if(re.sub(' "','"',foo) == changes[curkey] or re.sub('" ','"',foo) == changes[curkey] or re.sub('-','—',foo) == changes[curkey]):
                        l,hits = re.subn(startval,'<corr>'+changes[curkey]+'</corr>',l)
                    else:
                        #print(foo)
                        l,hits = re.subn(startval,'<choice><sic>'+foo+'</sic><corr>'+changes[curkey]+'</corr></choice>',l)
                    #print(hits)
                    if(hits):
                        #print(startval,'['+foo+']',changes[curkey])
                        #print(l)
                        if(curkey in madechanges):
                            madechanges[curkey] = madechanges[curkey] + hits
                        else:
                            madechanges[curkey] = hits
                        
                    #print('retrieved',curkey,changes[curkey])
                else:
                    print('failed',curkey)
        
      
            
        print(l,file=outf)
    for foo in changes:
        if(not foo in madechanges):
            print('failed',foo)
            continue
        if(madechanges[foo]>1):
            print('ambig',foo,changes[foo],madechanges[foo])
    print(workn,curwords,len(changes),1-len(changes)/curwords)
    totwords = curwords + totwords
    totchanges = totchanges + len(changes)

for i in range(1,5):
    curwork = str(i)
    #print('doing',curwork)
    compvers(curwork)

#print(totwords,totchanges,1-totchanges/totwords)

diff -y /Users/gcrane/scratch/lucian/fowler-sac/words1.txt words2-1.txt > engdiff-1.txt
failed 187 burnt-offerings .
1 76823 436 0.9943246163258399
diff -y /Users/gcrane/scratch/lucian/fowler-sac/words2.txt words2-2.txt > engdiff-2.txt
2 92660 473 0.9948953162097993
diff -y /Users/gcrane/scratch/lucian/fowler-sac/words3.txt words2-3.txt > engdiff-3.txt
failed 100 not) —;
failed 168 — —?
failed 169 — —?
3 93448 659 0.9929479496618441
diff -y /Users/gcrane/scratch/lucian/fowler-sac/words4.txt words2-4.txt > engdiff-4.txt
4 62368 382 0.9938750641354541
